# UnFound NLP Assignment 

1. Given some input word or phrase, figure out the relevant Wikipedia article. If there’s
nothing relevant, return null.
2. Create a timeline from the article. To do that, extract all sentences with dates (&
temporal words like yesterday, tomorrow, etc). Return only best n (NOT necessarily first
n) sentences in timeline in chronological order. [we leave it up to how you infer best]
3. For each sentence in the timeline, output a sentence embedding. Choose an embedding
of your choice (Word2Vec, Glove, Universal Sentence Encoder, ELMo, etc).
4. Create a microservice (BONUS: host it on Heroku or similar) which takes some
word/phrase and a number n as input, & returns these
a. Relevant Wikipedia page name (return null if irrelevant)
b. Best n timeline sentences (with sentence embedding)

# For First Task
    1. Import required libraries wikipedia,wikipediaapi
    2. Accept User Input for query
    3. Search Relevant query

# 1.1 Importing Libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import wikipedia
import wikipediaapi

%matplotlib inline

# 1.2 Take input from User


In [4]:
article=input()

fifa world cup


#### 1.2.1  Processing the query before fetching results

In [5]:
import string
article=string.capwords(article)

article

'Fifa World Cup'

# 1.3 Finding article related to query

   #### 1.3.1 Storing page content if exists

In [6]:
wiki_wiki = wikipediaapi.Wikipedia('en')

page_py = wiki_wiki.page(article)
if(page_py.exists()):
    print("Page - Summary: %s" % page_py.text)
    page_content=page_py.text
else :
    print("NULL")

Page - Summary: The FIFA World Cup, often simply called the World Cup, is an international association football competition contested by the senior men's national teams of the members of the Fédération Internationale de Football Association (FIFA), the sport's global governing body. The championship has been awarded every four years since the inaugural tournament in 1930, except in 1942 and 1946 when it was not held because of the Second World War. The current champion is France, which won its second title at the 2018 tournament in Russia.
The current format of the competition involves a qualification phase, which currently takes place over the preceding three years, to determine which teams qualify for the tournament phase, which is often called the World Cup Finals. After this, 32 teams, including the automatically qualifying host nation(s), compete in the tournament phase for the title at venues within the host nation(s) over a period of about a month.
The 21 World Cup tournaments h

In [7]:
page_content

'The FIFA World Cup, often simply called the World Cup, is an international association football competition contested by the senior men\'s national teams of the members of the Fédération Internationale de Football Association (FIFA), the sport\'s global governing body. The championship has been awarded every four years since the inaugural tournament in 1930, except in 1942 and 1946 when it was not held because of the Second World War. The current champion is France, which won its second title at the 2018 tournament in Russia.\nThe current format of the competition involves a qualification phase, which currently takes place over the preceding three years, to determine which teams qualify for the tournament phase, which is often called the World Cup Finals. After this, 32 teams, including the automatically qualifying host nation(s), compete in the tournament phase for the title at venues within the host nation(s) over a period of about a month.\nThe 21 World Cup tournaments have been wo

#  2 .Select best n lines from the article (having dates and temporal words) in  chronological order
    1. Text Cleaning
    2. Split page line by line
    3. Creating a DataFrame to store relevant values
    4. Data Preprocessing
    5. Take input N 
    6. Select sentence according to ranking and chronological order
    

## 2.1 Text Cleaning
### 2.1.1 Remove \n from article




In [8]:
page_content=page_content.replace('\n',' ')

### 2.1.2 Remove abbrevated text
   #### - usually consist of name initials in the form of [A-Z]. 

In [9]:
import re
page_content=re.sub(r"\b[A-Z\.]\b"," ",page_content)

## 2.2 Split page line by line

In [10]:
page_lines=page_content.split('.')
page_lines

["The FIFA World Cup, often simply called the World Cup, is an international association football competition contested by the senior men's national teams of the members of the Fédération Internationale de Football Association (FIFA), the sport's global governing body",
 ' The championship has been awarded every four years since the inaugural tournament in 1930, except in 1942 and 1946 when it was not held because of the Second World War',
 ' The current champion is France, which won its second title at the 2018 tournament in Russia',
 ' The current format of the competition involves a qualification phase, which currently takes place over the preceding three years, to determine which teams qualify for the tournament phase, which is often called the World Cup Finals',
 ' After this, 32 teams, including the automatically qualifying host nation(s), compete in the tournament phase for the title at venues within the host nation(s) over a period of about a month',
 ' The 21 World Cup tournam

### Updating  page content and page lines

In [11]:
# Update after testing
page_content=re.sub(r"\s\.","",page_content)
#updated split by lines
page_lines=page_content.split('.')
page_lines

["The FIFA World Cup, often simply called the World Cup, is an international association football competition contested by the senior men's national teams of the members of the Fédération Internationale de Football Association (FIFA), the sport's global governing body",
 ' The championship has been awarded every four years since the inaugural tournament in 1930, except in 1942 and 1946 when it was not held because of the Second World War',
 ' The current champion is France, which won its second title at the 2018 tournament in Russia',
 ' The current format of the competition involves a qualification phase, which currently takes place over the preceding three years, to determine which teams qualify for the tournament phase, which is often called the World Cup Finals',
 ' After this, 32 teams, including the automatically qualifying host nation(s), compete in the tournament phase for the title at venues within the host nation(s) over a period of about a month',
 ' The 21 World Cup tournam

## 2.3 Create a DataFrame to store all sentences

In [12]:
df=pd.DataFrame({'Sentence':page_lines})

In [13]:
df.describe()

,Sentence
count,203
unique,201
top,
freq,3


### 2.3.1 Form a year column

In [14]:
# Forming a year column in dataframe
Years_line=[]
for x in page_lines:
    l=re.search(r"(\d{4})",x)
    if l:
        h=int(l.group(0))
        if h<2099:
            Years_line.append(h)
        else:
            Years_line.append(None)
    else:
        Years_line.append(None)
df['Year']=Years_line
df   

,Sentence,Year
0,"The FIFA World Cup, often simply called the Wo...",NaN
1,The championship has been awarded every four ...,1930.0
2,"The current champion is France, which won its...",2018.0
3,The current format of the competition involve...,NaN
4,"After this, 32 teams, including the automatic...",NaN
5,The 21 World Cup tournaments have been won by...,NaN
6,"Brazil have won five times, and they are the ...",NaN
7,The other World Cup winners are Germany and I...,NaN
8,The World Cup is the most prestigious associa...,2006.0
9,"Brazil, France, Italy, Germany and Mexico hav...",NaN


In [15]:
df.describe()

,Year
count,106.000000
mean,1970.962264
std,38.283137
min,1872.000000
25%,1934.000000
50%,1976.000000
75%,2006.000000
max,2026.000000


In [16]:
MONTHS_PATTERN = ['january','february','march','april','may','june','july','august','september','october','november','december','jan','feb','mar','apr','may','jun','jul','aug','sep','sept','oct','nov','dec','today','tomorrow','yesterday']

### 2.3.2 Add new Column in Dataframe for months/temporal words detection
#### 1- indicates month/temporal words  are present

In [17]:
dpset=set()
for num,line in enumerate(page_lines):
    l=re.search(r"(\d{4})",line)
    if l:
        h=l.group(0)
        if int(h)<2099:
            dpset.add(num)
    words=line.split(' ')
    for wn,word in enumerate(words):
        if word.lower() in MONTHS_PATTERN:
            dpset.add(num)
for x in dpset:
    df.at[x, 'Date Present']=1

df

,Sentence,Year,Date Present
0,"The FIFA World Cup, often simply called the Wo...",NaN,NaN
1,The championship has been awarded every four ...,1930.0,1.0
2,"The current champion is France, which won its...",2018.0,1.0
3,The current format of the competition involve...,NaN,NaN
4,"After this, 32 teams, including the automatic...",NaN,NaN
5,The 21 World Cup tournaments have been won by...,NaN,NaN
6,"Brazil have won five times, and they are the ...",NaN,NaN
7,The other World Cup winners are Germany and I...,NaN,NaN
8,The World Cup is the most prestigious associa...,2006.0,1.0
9,"Brazil, France, Italy, Germany and Mexico hav...",NaN,NaN


## 2.4 Data preprocessing
### 2.4.1 Drop rows with NA values

In [18]:
df.dropna(inplace=True)
df=df.reset_index(drop=True)
df

,Sentence,Year,Date Present
0,The championship has been awarded every four ...,1930.0,1.0
1,"The current champion is France, which won its...",2018.0,1.0
2,The World Cup is the most prestigious associa...,2006.0,1.0
3,Qatar are planned as hosts of the 2022 finals...,2022.0,1.0
4,History Previous international competitions ...,1872.0,1.0
5,"The first international tournament, the inaug...",1884.0,1.0
6,As football grew in popularity in other parts...,1900.0,1.0
7,These were very early days for international ...,1908.0,1.0
8,They repeated the feat at the 1912 Summer Oly...,1912.0,1.0
9,With the Olympic event continuing to be conte...,1909.0,1.0


## To sort Sentences in Chronological Order

In [33]:
df=df.sort_index(by='Year')

C:\Users\Pranshu Die\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


### 2.4.2 Best Sentences 

#### 2.4.2.1 Calculate Number of rows

In [34]:
# Store Value of number of rows
nr=df.shape[0]

#### 2.4.2.2 Forming a Vector representaion of the best sentences using doc2vec model 

#### 2.5.2.2.1 Calculating similarity and giving ranks to sentences 

In [35]:
from gensim.models import doc2vec
from collections import namedtuple

# Load data

doc1 =df['Sentence']

# Transform data (you can add more data preprocessing steps) 

docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(doc1):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

# Train model (set min_count = 1, if you want the model to work with the provided example data set)

model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)


tokens =article.split()

new_vector = model.infer_vector(tokens)
sim_list = model.docvecs.most_similar([new_vector],topn=nr) 

C:\Users\Pranshu Die\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


#### Giving rank to each sentence 

In [39]:
#Rank to each sentence
rank=[i[0] for i in sim_list]

In [40]:
df['Ranking']=rank

## 2.5 Taking input for N for best lines

In [41]:
# Number of best Lines

no_best_lines=input()

8


## 2.6 Selecting n best sentences and ordering then in chronological order

In [42]:
nbest_lines=list()
nyear=list()
for index, row in df.iterrows():
    if row['Ranking'] < int(no_best_lines):
        print (row['Year'],':', row['Sentence'])
        nbest_lines.append(row['Sentence'])
        nyear.append(row['Year'])

1884.0 :  The first international tournament, the inaugural British Home Championship, took place in 1884
1930.0 :  Another match or matches drew more attendance than the final in 1930, 1938, 1958, 1962, 1970–1982, 1990 and 2006
1934.0 :  Italy's Vittorio Pozzo is the only head coach to ever win two World Cups (1934 and 1938)
1958.0 :  The fourth placed goalscorer, France's Just Fontaine, holds the record for the most goals scored in a single World Cup; all his 13 goals were scored in the 1958 tournament In November 2007, FIFA announced that all members of World Cup-winning squads between 1930 and 1974 were to be retroactively awarded winners' medals
1970.0 :  Brazil were also the first team to win the World Cup for the third (1970), fourth (1994) and fifth (2002) time
1970.0 :  West Germany's Gerd Müller (1970–1974) is third, with 14 goals
1991.0 :   Other FIFA tournaments An equivalent tournament for women's football, the FIFA Women's World Cup, was first held in 1991 in China
2017.0

# 3. For each sentence in the timeline, output a sentence embedding. Choose an embedding of your choice.

        1. Converting sentences to vectors
        2. Output sentences in Correct order

## 3.1 Converting the best n sentences to vectors using doc2vec

In [43]:
# Sentence Embedding 


nbest_docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(nbest_lines):
    words = text.lower().split()
    tags = [i]
    nbest_docs.append(analyzedDocument(words, tags))

In [44]:
# Model to Convert Best sentences to Vector format

best_model = doc2vec.Doc2Vec(nbest_docs, size = 100, window = 300, min_count = 1, workers = 4)

C:\Users\Pranshu Die\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


## 3.2 Displaying Sentences in order with chronological order with Sentence Embedding

In [62]:
# Displaying N best sentences with Sentence Embedding
for num ,i in enumerate (best_model.docvecs):
    if num<int(no_best_lines):
        print(nyear[num],':',nbest_lines[num])
        print(best_model.docvecs[num])


1884.0 :  The first international tournament, the inaugural British Home Championship, took place in 1884
[ 8.1368728e-04 -4.4577490e-03 -2.8431183e-04  1.0566044e-03
 -4.1513323e-04 -2.3208999e-03  4.2975275e-03 -2.3911141e-03
 -1.4077462e-03 -4.2437576e-03 -2.4414419e-03  8.7947847e-04
 -1.5177766e-03  2.1510779e-04 -1.6571224e-03  1.8568553e-03
  4.4119051e-03  3.0719170e-03  3.9545088e-03  4.5753042e-03
 -4.7163656e-03  3.8739445e-04  1.6786375e-03  3.9810771e-03
 -4.3969941e-03 -1.5532423e-03  4.2679198e-03 -3.9132740e-03
 -8.6513930e-04  1.8630541e-03 -4.3592025e-03 -3.1838845e-03
  1.4295252e-03  1.0735038e-03  1.3995814e-03 -2.8406854e-03
 -3.6788299e-03 -1.2160244e-03  2.8107606e-03 -4.6570338e-03
  2.9574675e-04  2.5856479e-03 -8.5758063e-04  2.0008965e-03
 -2.9937592e-03  3.9592801e-04 -4.2148987e-03 -4.6707243e-03
  2.5170073e-03 -4.4756774e-03  3.9921380e-03  2.6931698e-03
  2.7630385e-03  3.6103318e-03  4.4388091e-03 -2.0679780e-03
  1.9263367e-03 -2.8692367e-03 -3.001418

KeyError: "tag '8' not seen in training corpus/invalid"

### Conclusion:
In this assignmnet I have tried to achieve the required goal.The results from the notebook satisfy all the required pointers,It prints the best n sentences with respect to the given article, with sentence embedding of each sentence

## I would like to know your improvements and suggestions on the same